## CatBoost Tabular Prediction (Oct 2021)
In this Notebook, I will:
* Build a Tabular Prediciton Model using Catboost.
* Make this Notebook available to train on Google Colab or Other Platfroms or your Computer. (It may need to adjust code and configuration), The Competition result can be submited automatically.

## Setup

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
try:
  from catboost import CatBoostClassifier
except:
  !pip install catboost
  from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import os

In [ ]:
class Config:
    is_kaggle_platform = os.path.exists("/kaggle/input")
    dataset_name = "tabular-playground-series-oct-2021"
    data_path = "/kaggle/input/%s/"%(dataset_name) if is_kaggle_platform else ""
    submit_filename = "submission.csv"
config = Config()

In [ ]:
if not config.is_kaggle_platform:
  try:
    import kaggle
  except:
    !pip install kaggle
  if not os.path.exists("/root/.kaggle/kaggle.json"):
    !echo "{\"username\":\"{your username}\",\"key\":\"{your api key}\"}" >> /root/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json
  !kaggle competitions download -c $config.dataset_name
  !unzip test.csv.zip
  !unzip train.csv.zip
  !unzip sample_submission.csv.zip

In [ ]:
train = pd.read_csv(config.data_path + "train.csv")
test = pd.read_csv(config.data_path + "test.csv")
sample_submission = pd.read_csv(config.data_path + "sample_submission.csv")

## EDA

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train_features, val_features = train_test_split(train, test_size=0.15, random_state=42)

In [ ]:
val_features.shape

In [ ]:
train_targets = train_features.pop("target")
val_targets = val_features.pop("target")

## Model Development

In [ ]:
cat_params = {
    'iterations': 15585, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 7, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 1000,
    'eval_metric': 'AUC',
    'use_best_model': True
}
cat = CatBoostClassifier(**cat_params)
cat.fit(train_features, train_targets, eval_set=(val_features, val_targets))

## Submission

In [ ]:
y_pred = cat.predict_proba(test)
sample_submission["target"] = y_pred[:, 1]
sample_submission.to_csv(config.submit_filename, index=False)
if not config.is_kaggle_platform:
  !kaggle competitions submit $config.dataset_name -m "Submission" -f $config.submit_filename